# Imports

In [1]:
import numpy as np
import skimage
import matplotlib.cm as cm
from tqdm.notebook import tqdm
from numba import jit
import math
from PIL import Image

Für `.png` Dateien den oberen Block auführen. Bilder vom `.tiff`-Typ müssen mit dem Block darunter eingelesen werden.

In [ ]:
# PNG image
orig_image = skimage.io.imread("images_test/lenna_2048.png",
                               as_gray=True)
skimage.io.imshow(orig_image, cmap=cm.gray)

In [ ]:
# TIFF image
orig_image = Image.open("images_test/CodA_381_KTK_colorUV.tif")
orig_image = np.array(orig_image)

skimage.io.imshow(orig_image)

# VII

In [ ]:
r_vii = 35

a = orig_image.shape[0]+(r_vii*2)
b = orig_image.shape[1]+(r_vii*2)

image = np.full((a, b), 255)

c = int(image.shape[0]-r_vii)
d = int(image.shape[1]-r_vii)

image[r_vii:c, r_vii:d] = orig_image

@jit(nopython=True)
def volume_integral_invariant(img, radius, x, y):
    volume = 0

    def calc_h(i, j):
        if radius ** 2 - x ** 2 - y ** 2 >= 0:
            return img[abs(y + j), abs(x + i)] - img[y, x] + np.sqrt(radius ** 2 - x ** 2 - y ** 2)
        else:
            return img[abs(y + j), abs(x + i)] - img[y, x]

    def calc_sphere(i, j):
        if radius ** 2 - i ** 2 - j ** 2 >= 0:
            return 2 * np.sqrt(radius ** 2 - i ** 2 - j ** 2)
        else:
            return 0

    for i in range(-radius, radius + 1):
        for j in range(-radius, radius + 1):

            h = calc_h(i, j)
            sphere = calc_sphere(i, j)

            tmp = min(h, sphere)
            volume += max(tmp, 0)
    # normalizing
    if volume != 0:
        volume = (3 * volume) / (2 * np.pi * radius ** 3) - 1
    return volume


def vii_image(input_img, radius):
    empty = np.ones((input_img.shape[0]-(2*radius), input_img.shape[1]-(2*radius)))
    print("Calculating volume")
    for y in tqdm(range(0, empty.shape[0])):
        for x in range(0, empty.shape[1]):
            empty[y, x] = volume_integral_invariant(input_img, radius, x+radius, y+radius)
    return empty


volume_image = vii_image(image, r_vii)
skimage.io.imshow(volume_image, cmap=cm.gray)

In [ ]:
skimage.io.imsave(f"volume_lenna_r_vii{r_vii}.png", volume_image)

# SII

In [ ]:
r_sii = math.floor(r_vii/2) #paper seite 92
epsilon = 0.1

In [ ]:
@jit(nopython=True)
def is_zero_crossing(volume, x, y):
    for i in range(-1, 2):
        for j in range(-1, 2):
            if (i != 0 or j != 0):
              if (volume.shape[0] > y + i >= 0 and
                      0 <= x + j < volume.shape[1]):
                  if (volume[y + i, x + j] * volume[y, x] < 0 or
                          (volume[y + i, x + j] == 0 and volume[y, x] < 0)):
                      return 0
    return 1


def zero_crossing(volume):
    empty = np.zeros((volume.shape[0], volume.shape[1]))
    print("Check for zero crossings")
    for y in tqdm(range(0, empty.shape[0])):
        for x in range(0, empty.shape[1]):
            empty[y, x] = is_zero_crossing(volume, x, y)
    return empty


zero_crossings = zero_crossing(volume_image)
skimage.io.imshow(zero_crossings)

In [ ]:
skimage.io.imsave(f"zerocross_grid_r_vii{r_vii}_r_sii{r_sii}.png", zero_crossings)

In [ ]:
@jit(nopython=True)
def patch(input_img, scharr_img, radius, x, y):
    p = 0
    for i in range(-radius, radius + 1):
        for j in range(-radius, radius + 1):
            if 0 <= y + i < input_img.shape[0] and 0 <= x + j < input_img.shape[1]:
                if (input_img[y + i, x + j] - input_img[y, x]) ** 2 + i ** 2 + j ** 2 <= radius ** 2:
                    p += scharr_img[y + i, x + j]
    return p


@jit(nopython=True)
def min_patch(patch, radius, x, y):
    if x - radius < 0:
        a = 0
        b = x + radius + 1
    elif x + radius >= patch.shape[1]:
        a = x - radius
        b = patch.shape[1]
    else:
        a = x - radius
        b = x + radius + 1
    if y - radius < 0:
        c = 0
        d = y + radius + 1
    elif y + radius >= patch.shape[0]:
        c = y - radius
        d = patch.shape[0]
    else:
        c = y - radius
        d = y + radius + 1
    #print(f"x = {x}, y = {y}")
    #print(patch[c:d, a:b].shape)
    return min(map(min, patch[c:d, a:b]))


def t_mask(input_image, radius):
    scharr_img = skimage.filters.scharr(input_image)
    empty = np.zeros((input_image.shape[0]-2*r_vii, input_image.shape[1]-2*r_vii))
    #print(f"empty = {empty.shape}")
    t = np.zeros(input_image.shape)
    print("Create patch")
    for y in tqdm((range(0, empty.shape[0]))):
        for x in range(0, empty.shape[1]):
            empty[y, x] = patch(input_image, scharr_img, radius, x+r_vii, y+r_vii)
    #skimage.io.imshow(empty)
    print("Calculate t-mask")
    for y in tqdm(range(0, empty.shape[0])):
        for x in range(0, empty.shape[1]):
            tmp = min_patch(empty, radius, x, y) * (2 + epsilon)
            if empty[y, x] > tmp:
                t[y, x] = 1
    return t

t = t_mask(image, r_sii)

In [ ]:
def is_edge(zero_crossings, t, x, y):
    return zero_crossings[y, x] * t[y, x]


def edge_image(zero_crossings, t):
    empty = np.zeros(zero_crossings.shape)
    print("Calculate edge image")
    #print(zero_crossings.shape)
    #print(t.shape)
    for y in tqdm(range(0, zero_crossings.shape[0])):
        for x in range(0, zero_crossings.shape[1]):
            #print(f"x = {x}, y = {y}")
            empty[y, x] = is_edge(zero_crossings, t, x, y)
    return empty


edge_img = edge_image(zero_crossings, t)
skimage.io.imshow(edge_img)

In [ ]:
skimage.io.imsave(f"edge_grid_r_vii{r_vii}_r_sii{r_sii}.png", edge_img)

# Laufzeiten

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

runtimes_volume = pd.DataFrame(
    {"r3" : [12,7,8,8,6,6,8,7,6,5],
     "r11": [16,16,16,18,18,17,16,16,17,18],
     "r35": [125,123,122,121,122,124,120,123,124,122],
     "r51": [236,235,233,229,227,221,241,243,238,234]}
)

means = runtimes_volume.mean()

for_scatter = pd.melt(runtimes_volume, var_name="Radius", value_name="Laufzeit")
for_scatter.replace({"r3": 3, "r11": 11, "r35": 35, "r51": 51}, inplace=True)

plt.figure()
plt.plot([3,11,35,51],list(means))
plt.violinplot(runtimes_volume,
               positions=[3,11,35,51],
               widths=1,
               showmeans=True)
plt.xlabel("Radius")
plt.ylabel("Laufzeit in Sekunden")

plt.show()